## Script for jrn_studies tables

This will populate the public metadata table (public.qgis_layer_metadata) with
records for each layer in the jrn_studes schema. The qmd parts (qmd column) in
the table are not the most complete. It will be necessary to alter the qmd crs
and extents manually in QGIS.

In [1]:
# Import sqlalchemy and pandas
import sqlalchemy as sqla
import pandas as pd
import numpy as np
import geopandas as gp
import xml.etree.ElementTree as ET
from shapely.geometry import box

In [1]:
# Get credentials for the jgeo database
import sys
sys.path.append('/home/greg/admin/')
import jgeo_cred as dbcred

# get a connection to jgeo
conn = sqla.create_engine("postgresql+psycopg2://{0}:{1}@{2}:5432/{3}".format(dbcred.user, dbcred.pwd, dbcred.host, dbcred.db))

In [2]:
# Some functions

def update_md_record_id(current_id, gdf, schemaname, tablename):
    new_id = current_id
    current_id_fields = current_id.split(' ')
    new_id = new_id.replace(current_id_fields[6], 'srid={0}'.format(gdf.crs.to_epsg()))
    new_id = new_id.replace(current_id_fields[7], 'type={0}'.format(gdf.geom_type[1]))
    new_id = new_id.replace(current_id_fields[9], 'table="{0}"."{1}"'.format(schemaname, tablename))
    return(new_id)

def update_md_qmd(current_rec, gdf, schemaname, tablename, abstract):
    # Read the qmd into an elementtree
    root = ET.fromstring(current_rec.qmd.values[0])
    # Make some changes to the XML element contents
    root.find('identifier').text = update_md_record_id(current_rec.identifier.values[0],
                                                       gdf, schemaname,
                                                       tablename)
    root.find('parentidentifier').text = 'jgeo/{0}'.format(schemaname)
    root.find('language').text = 'en'
    # title is a litte different than tablename for jrn_studies
    if schemaname=='jrn_studies':
        root.find('title').text = tablename.split('_', 1)[1]
    else:
        root.find('title').text = tablename
    root.find('abstract').text = abstract
    # These are hard to fill in - leaving as is
    root.find('crs').text = ''
    root.find('extent').text = '' # Contains a "spatial" sub-element that could be changed
    return(ET.tostring(root, encoding='unicode'))

In [3]:
# The schema to make metadata files for
jgeo_schema = 'jrn_studies'

# Read the public.qgis_layer_metadata table into a dataframe
query = "select * from public.qgis_layer_metadata" 
df = pd.read_sql(query, conn)
#df.head()

# Get a template dataframe
template = df.iloc[-1:].copy()
print('Using template: ' + template.f_table_name)

# Get a list of current metadata records in public.qgis_layer_metadata
current_md = df.f_table_name
# Index to start at (max in the current metadata table)
id_start = df.id.max() + 1 # ID to start at

# Get a list of layers in the requested schema
insp = sqla.inspect(conn)
schema_tables = insp.get_table_names(schema=jgeo_schema)
# Get the set that don't already have a metadata entry
tables2add = np.setdiff1d(schema_tables,current_md)
# Exclude a few layers with geometries that won't read
# with fiona/geopandas
tables2add = np.setdiff1d(tables2add, ['prj185_el12_21_poly','prj185_erle_polygons',
                                       'prj185_iterburn','prj185_noburn84','prj228_neat_blk_1',
                                      'prj404_cdrrcvegetation1938','prj404_cdrrcvegetation1998',
                                      'prj430_site_locations','prj445_neon_insect_polys',
                                      'prj466_pitfalltraps_nppquadpolys','prj480_waterfeatures',
                                      'prj488_study_sites','prj499_transect'])

# Now loop through the tables and update 
for i, t in enumerate(tables2add[1:3]):
    print(t)
    # Make an abstract for the table and qmd
    abst = 'This is a placeholder abstract for the {0} geospatial layer (jgeo \
table name: {1}). This layer is part of the Jornada studies collection \
(jgeo/jrn_studies), which is a collection of geospatial layers describing \
research locations associated with the USDA-ARS Jornada Experimental \
Range and Jornada Basin LTER programs. This metadata was automatically \
generated from a template.'.format(t.split('_', 1)[1], t)
    
    # Read as geodataframe with geopandas
    gdf = gp.GeoDataFrame.from_postgis("SELECT * FROM {0}.{1};".format(jgeo_schema, t),
                                       conn, geom_col='geom', index_col='ogc_fid',
                                       coerce_float=False)
    # Create a new record dataframe and give the next id
    newrec = template.copy()
    newrec.id = id_start + i
    # Get the extent and convert to a WKT polygon
    bounds = gdf.total_bounds
    poly = box(*bounds)
    newrec.extent = poly.wkt
    # Replace schema and table names
    newrec.f_table_schema = jgeo_schema
    newrec.f_table_name = t
    # Replace identifier field
    newrec.identifier = update_md_record_id(newrec.identifier.values[0],
                                            gdf, jgeo_schema, t)
    # Replace title, abstract, other stuff
    newrec.title = t.split('_', 1)[1]
    newrec.abstract = abst
    newrec.geometry_type = gdf.geom_type[1]
    newrec.crs = 'EPSG:{0}'.format(gdf.crs.to_epsg())
    # Replace qmd file
    newrec.qmd = update_md_qmd(newrec, gdf, jgeo_schema, t, abst)
    
    if i==0:
        outdf = newrec
    else:
        outdf = pd.concat([outdf, newrec])
        

615    Using template: twdietower_wgs84
Name: f_table_name, dtype: object


In [90]:
outdf.head()

,id,extent,f_table_catalog,f_table_schema,f_table_name,f_geometry_column,identifier,title,abstract,geometry_type,crs,layer_type,qmd,owner,update_time
30,35,POLYGON ((-106.7923260259999 32.51155793600009...,jgeo,jrn_studies,prj001_hydrology_runoff_drums,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,hydrology_runoff_drums,This is a placeholder abstract for the hydrolo...,MultiPolygon,EPSG:4326,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
30,36,POLYGON ((-106.8456380699999 32.58595226500006...,jgeo,jrn_studies,prj001_hydrology_runoff_exclosures,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,hydrology_runoff_exclosures,This is a placeholder abstract for the hydrolo...,MultiPolygon,EPSG:4326,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
30,37,POLYGON ((-106.7923381229999 32.51153290800005...,jgeo,jrn_studies,prj001_hydrology_runoff_gps_points,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,hydrology_runoff_gps_points,This is a placeholder abstract for the hydrolo...,MultiPoint,EPSG:4326,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
30,38,POLYGON ((-106.7923401819999 32.51154456800009...,jgeo,jrn_studies,prj001_hydrology_runoff_plates,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,hydrology_runoff_plates,This is a placeholder abstract for the hydrolo...,MultiPolygon,EPSG:4326,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
30,39,POLYGON ((-106.7923411449999 32.51153156000004...,jgeo,jrn_studies,prj001_hydrology_runoff_plots,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,hydrology_runoff_plots,This is a placeholder abstract for the hydrolo...,MultiPolygon,EPSG:4326,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175


In [88]:
# Append to qgis_layer_metadata table
outdf.to_sql('qgis_layer_metadata', schema='public', con=conn, if_exists='append', index=False)

529

## Script for bounds_and_structures tables

This will populate the public metadata table (public.qgis_layer_metadata) with
records for each layer in the bounds_and_structures schema. The qmd parts (qmd column) in
the table are not the most complete. It will be necessary to alter the qmd crs
and extents manually in QGIS.

In [100]:
# The schema to make metadata files for
jgeo_schema = 'bounds_and_structures'

# Read the public.qgis_layer_metadata table into a dataframe
query = "select * from public.qgis_layer_metadata" 
df = pd.read_sql(query, conn)
#df.head()

# Get a template dataframe
template = df.iloc[-1:].copy()
print('Using template: ' + template.f_table_name)

# Get a list of current metadata records in public.qgis_layer_metadata
current_md = df.f_table_name
# Index to start at (max in the current metadata table)
id_start = df.id.max() + 1 # ID to start at

# Get a list of layers in the requested schema
insp = sqla.inspect(conn)
schema_tables = insp.get_table_names(schema=jgeo_schema)
# Get the set that don't already have a metadata entry
tables2add = np.setdiff1d(schema_tables,current_md)
# Exclude a few layers with geometries that won't read
# with fiona/geopandas
tables2add = np.setdiff1d(tables2add, ['random_troughs'])

# Now loop through the tables and update 
for i, t in enumerate(tables2add):
    print(t)
    # Make an abstract for the table and qmd
    abst = 'This is a placeholder abstract for the {0} geospatial layer (jgeo \
table name: {1}). This layer is part of the Jornada "boundaries and \
structures" collection(jgeo/bounds_and_structures), which is a collection \
of geospatial layers describing administrative boundaries and the built \
environment associated with the USDA-ARS Jornada Experimental Range (JER), \
Jornada Basin LTER program (JRN), and the Chihuahuan Desert Rangeland Research \
Center (CDRRC). This metadata was automatically generated from a template. \
 \
 These layers were derived from layers in the "Infrastructure2_21Sep15.gdb" \
 geodatabase.'.format(t, t)
    
    # Read as geodataframe with geopandas
    gdf = gp.GeoDataFrame.from_postgis("SELECT * FROM {0}.{1};".format(jgeo_schema, t),
                                       conn, geom_col='geom', index_col='ogc_fid',
                                       coerce_float=False)
    # Create a new record dataframe and give the next id
    newrec = template.copy()
    newrec.id = id_start + i
    # Get the extent and convert to a WKT polygon
    bounds = gdf.total_bounds
    poly = box(*bounds)
    newrec.extent = poly.wkt
    # Replace schema and table names
    newrec.f_table_schema = jgeo_schema
    newrec.f_table_name = t
    # Replace identifier field
    newrec.identifier = update_md_record_id(newrec.identifier.values[0],
                                            gdf, jgeo_schema, t)
    # Replace title, abstract, other stuff
    newrec.title = t #t.split('_', 1)[1]
    newrec.abstract = abst
    newrec.geometry_type = gdf.geom_type[1]
    newrec.crs = 'EPSG:{0}'.format(gdf.crs.to_epsg())
    # Replace qmd file
    newrec.qmd = update_md_qmd(newrec, gdf, jgeo_schema, t, abst)
    
    if i==0:
        outdf = newrec
    else:
        outdf = pd.concat([outdf, newrec])
        



559    Using template: prj518_uav_testsite_boundary
Name: f_table_name, dtype: object
airstrip
all3_bdry_25aug15
cattleguards
cdnp_bdry_25aug15
cdrrc_bdry_25aug15
cdrrc_excloedited21aug13
cdrrc_exclosures_17sep15
closedroad
csisparking
defuncttowers
dirt_tanks
doqq_list_jer
doqs_list_jer
exclosuresjer_17jul2015
exclusuresjer_jul172015
fence_cdrrc_26aug15
fence_jer_28aug15
gates
jer_bdry_25aug15
jer_cdrrc_bdry_fromcorrectedmar2014
jer_deededbndry_oct2013
jer_ftb_wsmr_bndry
lvstk_corral_lot_lane
offlimits_20aug15
offlimits_27feb2014_final
pastures_cdrrc2_16sep15
pastures_jer_16sep15
ranchstructures
rds_cliptojer_27apr18
remvdfencelabel
remvdfences
tanks
towers3
utep_site_access_trail_utm
wells


In [101]:
outdf.head()

,id,extent,f_table_catalog,f_table_schema,f_table_name,f_geometry_column,identifier,title,abstract,geometry_type,crs,layer_type,qmd,owner,update_time
559,564,"POLYGON ((337157.7632999998 3607722.1908, 3371...",jgeo,bounds_and_structures,airstrip,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,airstrip,This is a placeholder abstract for the airstri...,MultiLineString,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
559,565,"POLYGON ((358730.6323999995 3589721, 358730.63...",jgeo,bounds_and_structures,all3_bdry_25aug15,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,all3_bdry_25aug15,This is a placeholder abstract for the all3_bd...,MultiPolygon,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
559,566,"POLYGON ((350454.1290999996 3594464.2862, 3504...",jgeo,bounds_and_structures,cattleguards,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,cattleguards,This is a placeholder abstract for the cattleg...,MultiPoint,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
559,567,"POLYGON ((337005.8737000003 3593682.25, 337005...",jgeo,bounds_and_structures,cdnp_bdry_25aug15,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,cdnp_bdry_25aug15,This is a placeholder abstract for the cdnp_bd...,MultiPolygon,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
559,568,"POLYGON ((334001.7039999999 3589721, 334001.70...",jgeo,bounds_and_structures,cdrrc_bdry_25aug15,shape,dbname='jgeo' host=termite port=5432 sslmode=d...,cdrrc_bdry_25aug15,This is a placeholder abstract for the cdrrc_b...,MultiPolygon,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175


In [102]:
# Append to qgis_layer_metadata table
outdf.to_sql('qgis_layer_metadata', schema='public', con=conn, if_exists='append', index=False)

35

## Script for sensor_networks and other tables

This will populate the public metadata table (public.qgis_layer_metadata) with
records for each layer in the sensor_networks schema. The qmd parts (qmd column) in
the table are not the most complete. It will be necessary to alter the qmd crs
and extents manually in QGIS.

Basically the same for "biogeography_and_soils" and "physical_geography"

In [8]:
# The schema to make metadata files for
jgeo_schema = 'biogeography_and_soils'

# Read the public.qgis_layer_metadata table into a dataframe
query = "select * from public.qgis_layer_metadata" 
df = pd.read_sql(query, conn)
#df.head()

# Get a template dataframe
template = df.iloc[-1:].copy()
print('Using template: ' + template.f_table_name)

# Get a list of current metadata records in public.qgis_layer_metadata
current_md = df.f_table_name
# Index to start at (max in the current metadata table)
id_start = df.id.max() + 1 # ID to start at

# Get a list of layers in the requested schema
insp = sqla.inspect(conn)
schema_tables = insp.get_table_names(schema=jgeo_schema)
# Get the set that don't already have a metadata entry
tables2add = np.setdiff1d(schema_tables,current_md)
# Exclude a few layers with geometries that won't read
# with fiona/geopandas
#tables2add = np.setdiff1d(tables2add, ['random_troughs'])

# Now loop through the tables and update 
for i, t in enumerate(tables2add):
    print(t)
    # Make an abstract for the table and qmd
    abst = 'This is a placeholder abstract for the {0} geospatial layer (jgeo \
table name: {1}). This layer is part of the Jornada "biogeography and \
soils" collection (jgeo/biogeography_and_soils), which is a collection \
of geospatial layers describing the vegetation, soils, and \
other aspects of biogeography and earth surface features in the vicinity of \
the USDA-ARS Jornada Experimental Range (JER), \
Jornada Basin LTER program (JRN), and the Chihuahuan Desert Rangeland Research \
Center (CDRRC). This metadata was automatically generated from a template. \
 \
Many of these layers were derived from layers in the \
 "NonResearchLyrs" geodatabase.'.format(t, t)
    
    # Read as geodataframe with geopandas
    gdf = gp.GeoDataFrame.from_postgis("SELECT * FROM {0}.{1};".format(jgeo_schema, t),
                                       conn, geom_col='geom', index_col='ogc_fid',
                                       coerce_float=False)
    # Create a new record dataframe and give the next id
    newrec = template.copy()
    newrec.id = id_start + i
    # Get the extent and convert to a WKT polygon
    bounds = gdf.total_bounds
    poly = box(*bounds)
    newrec.extent = poly.wkt
    # Replace schema and table names
    newrec.f_table_schema = jgeo_schema
    newrec.f_table_name = t
    # Replace identifier field
    newrec.identifier = update_md_record_id(newrec.identifier.values[0],
                                            gdf, jgeo_schema, t)
    # Replace title, abstract, other stuff
    newrec.title = t #t.split('_', 1)[1]
    newrec.abstract = abst
    newrec.geometry_type = gdf.geom_type[1]
    newrec.crs = 'EPSG:{0}'.format(gdf.crs.to_epsg())
    # Replace qmd file
    newrec.qmd = update_md_qmd(newrec, gdf, jgeo_schema, t, abst)
    
    if i==0:
        outdf = newrec
    else:
        outdf = pd.concat([outdf, newrec])
        


607    Using template: twdietower_wgs84
Name: f_table_name, dtype: object
grass_cover_utm
jergrasscondition1858
jershrubpresence1858
jervegetation1915
jervegetation1928
jervegetation1998
soilmu_a_nm690
soilmu_a_nm719


In [9]:
outdf.head()

,id,extent,f_table_catalog,f_table_schema,f_table_name,f_geometry_column,identifier,title,abstract,geometry_type,crs,layer_type,qmd,owner,update_time
607,612,"POLYGON ((347042.9222339392 3605615.211546762,...",jgeo,biogeography_and_soils,grass_cover_utm,geom,dbname='jgeo' host=termite port=5432 sslmode=d...,grass_cover_utm,This is a placeholder abstract for the grass_c...,MultiPolygon,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
607,613,"POLYGON ((350830.1875 3594434.5, 350830.1875 3...",jgeo,biogeography_and_soils,jergrasscondition1858,geom,dbname='jgeo' host=termite port=5432 sslmode=d...,jergrasscondition1858,This is a placeholder abstract for the jergras...,MultiPolygon,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
607,614,"POLYGON ((350830.1875 3594434.5, 350830.1875 3...",jgeo,biogeography_and_soils,jershrubpresence1858,geom,dbname='jgeo' host=termite port=5432 sslmode=d...,jershrubpresence1858,This is a placeholder abstract for the jershru...,MultiPolygon,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
607,615,"POLYGON ((350914.8437000001 3590998.75, 350914...",jgeo,biogeography_and_soils,jervegetation1915,geom,dbname='jgeo' host=termite port=5432 sslmode=d...,jervegetation1915,This is a placeholder abstract for the jervege...,MultiPolygon,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175
607,616,"POLYGON ((348393.1699999999 3601386.119999999,...",jgeo,biogeography_and_soils,jervegetation1928,geom,dbname='jgeo' host=termite port=5432 sslmode=d...,jervegetation1928,This is a placeholder abstract for the jervege...,MultiPolygon,EPSG:32613,vector,"<qgis version=""3.30.1-'s-Hertogenbosch"">\n <i...",gmaurer,2023-01-06 16:54:05.175


In [10]:
# Append to qgis_layer_metadata table
outdf.to_sql('qgis_layer_metadata', schema='public', con=conn, if_exists='append', index=False)

8

In [129]:
root = ET.fromstring(newrec.qmd)
print(root.tag, root.attrib)
for child in root:
    print(child.tag, child.attrib)

qgis {'version': "3.30.1-'s-Hertogenbosch"}
identifier {}
parentidentifier {}
language {}
type {}
title {}
abstract {}
keywords {'vocabulary': 'GEMET'}
keywords {'vocabulary': 'gmd:topicCategory'}
contact {}
links {}
history {}
dates {}
fees {}
rights {}
license {}
encoding {}
crs {}
extent {}
